### Upload batch file

In [1]:
from dotenv import load_dotenv
import os
from openai import AzureOpenAI

load_dotenv()

client = AzureOpenAI(
    azure_endpoint = os.getenv("AZURE_OPENAI_API_ENDPOINT"),
    api_key=os.getenv("AZURE_OPENAI_API_KEY"),  
    api_version="2024-10-01-preview"
    )

# Upload a file with a purpose of "batch"
file = client.files.create(
  file=open("batch - StructuredOutputs.jsonl", "rb"), 
  purpose="batch"
)

print(file.model_dump_json(indent=2))
file_id = file.id

{
  "id": "file-72875f81636342009d1766fa429289eb",
  "bytes": 2022,
  "created_at": 1729518572,
  "filename": "batch - StructuredOutputs.jsonl",
  "object": "file",
  "purpose": "batch",
  "status": "pending",
  "status_details": null
}


### Track file upload status

In [2]:
# Wait until the uploaded file is in processed state
import time
import datetime 

status = "pending"
while status != "processed":
    time.sleep(15)
    file_response = client.files.retrieve(file_id)
    status = file_response.status
    print(f"{datetime.datetime.now()} File Id: {file_id}, Status: {status}")

2024-10-21 14:49:54.541926 File Id: file-72875f81636342009d1766fa429289eb, Status: processed


### Create batch job

In [3]:
# Submit a batch job with the file
batch_response = client.batches.create(
    input_file_id=file_id,
    endpoint="/chat/completions",
    completion_window="24h",
)

# Save batch ID for later use
batch_id = batch_response.id

print(batch_response.model_dump_json(indent=2))

{
  "id": "batch_6dcbe0ef-1c28-4477-a517-0c59b63ea107",
  "completion_window": "24h",
  "created_at": 1729518605,
  "endpoint": "/chat/completions",
  "input_file_id": "file-72875f81636342009d1766fa429289eb",
  "object": "batch",
  "status": "validating",
  "cancelled_at": null,
  "cancelling_at": null,
  "completed_at": null,
  "error_file_id": null,
  "errors": null,
  "expired_at": null,
  "expires_at": 1729605005,
  "failed_at": null,
  "finalizing_at": null,
  "in_progress_at": null,
  "metadata": null,
  "output_file_id": null,
  "request_counts": {
    "completed": 0,
    "failed": 0,
    "total": 0
  }
}


### Track batch job progress

In [4]:
import time
import datetime 

status = "validating"
while status not in ("completed", "failed", "canceled"):
    time.sleep(60)
    batch_response = client.batches.retrieve(batch_id)
    status = batch_response.status
    print(f"{datetime.datetime.now()} Batch Id: {batch_id},  Status: {status}")

2024-10-21 14:51:23.580562 Batch Id: batch_6dcbe0ef-1c28-4477-a517-0c59b63ea107,  Status: validating
2024-10-21 14:52:24.030281 Batch Id: batch_6dcbe0ef-1c28-4477-a517-0c59b63ea107,  Status: validating
2024-10-21 14:53:24.521352 Batch Id: batch_6dcbe0ef-1c28-4477-a517-0c59b63ea107,  Status: in_progress
2024-10-21 14:54:24.951208 Batch Id: batch_6dcbe0ef-1c28-4477-a517-0c59b63ea107,  Status: in_progress
2024-10-21 14:55:25.419307 Batch Id: batch_6dcbe0ef-1c28-4477-a517-0c59b63ea107,  Status: in_progress
2024-10-21 14:56:25.880283 Batch Id: batch_6dcbe0ef-1c28-4477-a517-0c59b63ea107,  Status: in_progress
2024-10-21 14:57:26.330489 Batch Id: batch_6dcbe0ef-1c28-4477-a517-0c59b63ea107,  Status: finalizing
2024-10-21 14:58:26.765073 Batch Id: batch_6dcbe0ef-1c28-4477-a517-0c59b63ea107,  Status: completed


#### Examine job status details

In [5]:
print(batch_response.model_dump_json(indent=2))

{
  "id": "batch_6dcbe0ef-1c28-4477-a517-0c59b63ea107",
  "completion_window": "24h",
  "created_at": 1729518605,
  "endpoint": "/chat/completions",
  "input_file_id": "file-72875f81636342009d1766fa429289eb",
  "object": "batch",
  "status": "completed",
  "cancelled_at": null,
  "cancelling_at": null,
  "completed_at": 1729519078,
  "error_file_id": "file-e0da6ce7-180a-4740-8aaa-caf1dc7d007b",
  "errors": null,
  "expired_at": null,
  "expires_at": 1729605005,
  "failed_at": null,
  "finalizing_at": 1729518991,
  "in_progress_at": 1729518794,
  "metadata": null,
  "output_file_id": "file-9b0af9ff-1842-4061-9bd7-c521d6fbf45b",
  "request_counts": {
    "completed": 3,
    "failed": 0,
    "total": 3
  }
}


### Retrieve batch job output file

In [6]:
import json

response = client.files.content(batch_response.output_file_id)
raw_responses = response.text.strip().split('\n')  

for raw_response in raw_responses:  
    json_response = json.loads(raw_response)  
    formatted_json = json.dumps(json_response, indent=2)  
    print(formatted_json)

{
  "custom_id": "task-0",
  "response": {
    "body": {
      "choices": [
        {
          "content_filter_results": {
            "hate": {
              "filtered": false,
              "severity": "safe"
            },
            "self_harm": {
              "filtered": false,
              "severity": "safe"
            },
            "sexual": {
              "filtered": false,
              "severity": "safe"
            },
            "violence": {
              "filtered": false,
              "severity": "safe"
            }
          },
          "finish_reason": "stop",
          "index": 0,
          "logprobs": null,
          "message": {
            "content": "{\"BuildingName\":\"Empire State Building\",\"HeightInFeet\":\"1,454\",\"City\":\"New York City\",\"Country\":\"United States\"}",
            "role": "assistant"
          }
        }
      ],
      "created": 1729518853,
      "id": "chatcmpl-AKn9JvRpWcqqqQ9rhlTMoX3Wyybkf",
      "model": "gpt-4o-2024-08-0

### Additional batch commands

#### Cancel batch

In [ ]:
client.batches.cancel("batch_abc123") # set to your batch_id for the job you want to cancel

#### List batch

In [ ]:
client.batches.list()